In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('/home/tammy.kojima/Authorship-attribution/')
import feature_extraction as fe
from tqdm import tqdm
import pickle
import os
import ast
from sklearn.model_selection import train_test_split

/home/tammy.kojima/miniconda3/envs/meu_ambiente/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-30 13:20:33,741	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-30 13:20:35,764	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/tammy.kojima/miniconda3/envs/meu_ambiente/lib/python3.9/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
df_human = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_human.csv')
df_human = df_human.iloc[:80000]

In [3]:
# Tokeniza todos os textos primeiro
textos = df_human["text"].fillna("").tolist()
processed_texts = [fe.preprocess_text(texto) for texto in tqdm(textos, desc="Tokenizando textos")]

Tokenizando textos:   0%|                                                                                                        | 0/80000 [00:00<?, ?it/s][nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
Tokenizando textos: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 80000/80000 [00:04<00:00, 16512.34it/s]


In [4]:
lexical_features = [
    fe.extract_lexical_features(textos[i], processed_texts[i])
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_lexical = pd.DataFrame(lexical_features)
df_test = pd.concat([df_human, df_lexical], axis=1)
df_test.head()

Extraindo features estilísticas: 100%|████████████████████████████████████████████████████████████████████████████| 80000/80000 [00:01<00:00, 69220.13it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio
0,@hlooman Hans I'm an open book you can ask me ...,human,NaN,0.838710,31,26,3.483871,2.701353,0.290323
1,@DavidArchie Take care. Hope to see you again ...,human,NaN,0.900000,20,18,4.000000,6.700000,0.150000
2,or apparently it just magically worked this ti...,human,NaN,1.000000,18,18,4.833333,6.805556,0.222222
3,* * * this was almost perfect weekend with a p...,human,NaN,0.750000,16,12,3.812500,6.152344,0.250000
4,"RALLY INFRONT OF MC?! Yup, I'm also scared abo...",human,NaN,0.954545,22,21,3.363636,3.231405,0.090909


In [5]:
stylistic_features = [
    fe.extract_stylistic_features(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_stylistic = pd.DataFrame(stylistic_features)
df_test = pd.concat([df_test, df_stylistic], axis=1)
df_test.head()

Extraindo features estilísticas: 100%|███████████████████████████████████████████████████████████████████████████| 80000/80000 [00:00<00:00, 118478.31it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density
0,@hlooman Hans I'm an open book you can ask me ...,human,NaN,0.838710,31,26,3.483871,2.701353,0.290323,0,0,0.000000,0.000000,0,0.0,0.0
1,@DavidArchie Take care. Hope to see you again ...,human,NaN,0.900000,20,18,4.000000,6.700000,0.150000,0,0,0.000000,0.000000,0,0.0,0.0
2,or apparently it just magically worked this ti...,human,NaN,1.000000,18,18,4.833333,6.805556,0.222222,0,0,0.000000,0.000000,0,0.0,0.0
3,* * * this was almost perfect weekend with a p...,human,NaN,0.750000,16,12,3.812500,6.152344,0.250000,0,0,0.000000,0.000000,0,0.0,0.0
4,"RALLY INFRONT OF MC?! Yup, I'm also scared abo...",human,NaN,0.954545,22,21,3.363636,3.231405,0.090909,0,0,0.045455,0.045455,0,0.0,0.0


In [6]:
structural_features = [
    fe.extract_structural_features(textos[i], processed_texts[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estruturais")
]
df_structural = pd.DataFrame(structural_features)
df_test = pd.concat([df_test, df_structural], axis=1)
df_test.head()

Extraindo features estruturais: 100%|████████████████████████████████████████████████████████████████████████████| 80000/80000 [00:00<00:00, 133504.65it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,stylistic_emoticon_density,structural_has_url,structural_has_mention,structural_has_hashtag,structural_is_retweet,structural_url_density,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers
0,@hlooman Hans I'm an open book you can ask me ...,human,NaN,0.838710,31,26,3.483871,2.701353,0.290323,0,...,0.0,0,1,0,0,0.0,0.032258,0.0,0.0000,0.0
1,@DavidArchie Take care. Hope to see you again ...,human,NaN,0.900000,20,18,4.000000,6.700000,0.150000,0,...,0.0,0,1,0,0,0.0,0.050000,0.0,0.0000,0.0
2,or apparently it just magically worked this ti...,human,NaN,1.000000,18,18,4.833333,6.805556,0.222222,0,...,0.0,0,0,0,0,0.0,0.000000,0.0,0.0000,0.0
3,* * * this was almost perfect weekend with a p...,human,NaN,0.750000,16,12,3.812500,6.152344,0.250000,0,...,0.0,0,0,0,0,0.0,0.000000,0.0,0.0625,0.0
4,"RALLY INFRONT OF MC?! Yup, I'm also scared abo...",human,NaN,0.954545,22,21,3.363636,3.231405,0.090909,0,...,0.0,0,0,0,0,0.0,0.000000,0.0,0.0000,0.0


In [7]:
syntactic_features = [
    fe.extract_syntactic_features(tokens)
    for tokens in tqdm(processed_texts, desc="Extraindo features sintáticas")
]
df_syntactic = pd.DataFrame(syntactic_features)
df_test = pd.concat([df_test, df_syntactic], axis=1)
df_test.head()

Extraindo features sintáticas: 100%|████████████████████████████████████████████████████████████████████████████████| 80000/80000 [04:55<00:00, 270.34it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,@hlooman Hans I'm an open book you can ask me ...,human,NaN,0.838710,31,26,3.483871,2.701353,0.290323,0,...,0.032258,0.0,0.0000,0.0,2.116684,3.059355,30.000000,0.0,0.064516,0.064516
1,@DavidArchie Take care. Hope to see you again ...,human,NaN,0.900000,20,18,4.000000,6.700000,0.150000,0,...,0.050000,0.0,0.0000,0.0,2.029396,2.830940,5.666667,0.0,0.050000,0.200000
2,or apparently it just magically worked this ti...,human,NaN,1.000000,18,18,4.833333,6.805556,0.222222,0,...,0.000000,0.0,0.0000,0.0,2.212257,2.833213,8.500000,0.0,0.000000,0.055556
3,* * * this was almost perfect weekend with a p...,human,NaN,0.750000,16,12,3.812500,6.152344,0.250000,0,...,0.000000,0.0,0.0625,0.0,2.046739,2.523211,6.000000,0.0,0.000000,0.250000
4,"RALLY INFRONT OF MC?! Yup, I'm also scared abo...",human,NaN,0.954545,22,21,3.363636,3.231405,0.090909,0,...,0.000000,0.0,0.0000,0.0,2.212427,2.965016,9.000000,0.0,0.090909,0.227273


In [8]:
ngrams_df = fe.extract_ngrams_features(processed_texts, max_features=300)
ngrams_df.head()

,2014,2019,about,after,again,against,all,all the,already,also,...,world,would,yeah,year,years,yes,you,you are,you have,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Resetar os índices de ambos os DataFrames
df_test_reset = df_test.reset_index(drop=True)
ngrams_df_reset = ngrams_df.reset_index(drop=True)

# Agora concatenar
df_combined = pd.concat([df_test_reset, ngrams_df_reset], axis=1)
df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})
df_combined.fillna(0)
df_combined.head()

/tmp/ipykernel_3453280/3577768971.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,world,would,yeah,year,years,yes,you,you are,you have,your
0,@hlooman Hans I'm an open book you can ask me ...,0,NaN,0.838710,31,26,3.483871,2.701353,0.290323,0,...,0,0,0,0,0,0,3,0,0,0
1,@DavidArchie Take care. Hope to see you again ...,0,NaN,0.900000,20,18,4.000000,6.700000,0.150000,0,...,0,0,0,0,0,0,2,0,0,0
2,or apparently it just magically worked this ti...,0,NaN,1.000000,18,18,4.833333,6.805556,0.222222,0,...,0,0,0,0,0,0,1,1,0,0
3,* * * this was almost perfect weekend with a p...,0,NaN,0.750000,16,12,3.812500,6.152344,0.250000,0,...,0,0,0,0,0,0,0,0,0,0
4,"RALLY INFRONT OF MC?! Yup, I'm also scared abo...",0,NaN,0.954545,22,21,3.363636,3.231405,0.090909,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_combined.to_csv("/home/tammy.kojima/Authorship-attribution/df_pronto/df_human_com_features.csv", index=False)

In [11]:
df_combined = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_human_com_features.csv')
X_new_features = fe.extract_features_batch(textos, processed_texts = processed_texts)
df_features = pd.DataFrame(X_new_features)
df_combined = pd.concat([df_combined, df_features], axis=1)
df_combined.head()

,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,chatgpt_structured_output,chatgpt_overly_polite,chatgpt_disclaimer_density,chatgpt_assistant_patterns,mistral_self_ref,mistral_structured_density,mistral_technical_jargon,mistral_non_english_density,mistral_step_reasoning,mistral_low_ethical_disclaimers
0,@hlooman Hans I'm an open book you can ask me ...,0,NaN,0.838710,31,26,3.483871,2.701353,0.290323,0,...,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.0
1,@DavidArchie Take care. Hope to see you again ...,0,NaN,0.900000,20,18,4.000000,6.700000,0.150000,0,...,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.05,0.0
2,or apparently it just magically worked this ti...,0,NaN,1.000000,18,18,4.833333,6.805556,0.222222,0,...,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.0
3,* * * this was almost perfect weekend with a p...,0,NaN,0.750000,16,12,3.812500,6.152344,0.250000,0,...,0.1875,0.0,0.0,0.0,0.0,0.1875,0.0,0.0,0.00,0.0
4,"RALLY INFRONT OF MC?! Yup, I'm also scared abo...",0,NaN,0.954545,22,21,3.363636,3.231405,0.090909,0,...,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.0


In [12]:
df_combined.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_human_com_features2.csv', index=False)

In [2]:
df = pd.read_csv("/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features2.csv")
df_human = pd.read_csv("/home/tammy.kojima/Authorship-attribution/df_pronto/df_human_com_features2.csv")

In [9]:
df_novo = pd.concat([df, df_human], ignore_index=True)
df_novo

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,vote for,wait,watch,watching,wish,with modi,without,yeah,yes,you have
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202390,you comparison based the assumptions that modi...,0,0.967742,31,30,5.354839,5.906348,0.129032,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202391,choukidar means daythis modi days modi finishe...,0,0.641026,39,25,5.307692,3.084813,0.000000,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202392,madam you have agree modi didnt had any corrup...,0,0.900000,20,18,5.200000,5.860000,0.150000,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
202393,where 15lakhs from swiss smart city promised m...,0,1.000000,9,9,5.111111,1.876543,0.000000,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_novo = df_novo.drop_duplicates()
df_novo

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,vote for,wait,watch,watching,wish,with modi,without,yeah,yes,you have
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202390,you comparison based the assumptions that modi...,0,0.967742,31,30,5.354839,5.906348,0.129032,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202391,choukidar means daythis modi days modi finishe...,0,0.641026,39,25,5.307692,3.084813,0.000000,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202392,madam you have agree modi didnt had any corrup...,0,0.900000,20,18,5.200000,5.860000,0.150000,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
202393,where 15lakhs from swiss smart city promised m...,0,1.000000,9,9,5.111111,1.876543,0.000000,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_novo['origin'].value_counts()

origin
1    100000
0    100000
Name: count, dtype: int64

In [ ]:
df_novo.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features2.csv', index=False)

In [21]:
# Lista de prefixos para remover
prefixos = ['ai_', 'llm_', 'mistral_', 'chatgpt_', 'temporal_', 'complexity_']

# Encontrar colunas que começam com esses prefixos
colunas_para_remover = [col for col in df.columns 
                       if any(col.startswith(prefixo) for prefixo in prefixos)]

print(f"📊 Colunas a serem removidas: {len(colunas_para_remover)}")
print("Exemplos:", colunas_para_remover[:10])  # Mostra as primeiras 10

# Remover as colunas
df_limpo = df_novo.drop(colunas_para_remover, axis=1)
print(f"✅ Shape após remoção: {df_limpo.shape}")

📊 Colunas a serem removidas: 27
Exemplos: ['ai_perplexity_score', 'ai_repeated_ngrams', 'ai_topic_shifts', 'ai_safety_disclaimers', 'ai_hedging_language', 'complexity_subordinating_ratio', 'complexity_long_sentences', 'complexity_rare_words', 'complexity_syntactic_depth', 'temporal_recent_ratio']
✅ Shape após remoção: (200000, 436)


In [22]:
df_limpo

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,vote for,wait,watch,watching,wish,with modi,without,yeah,yes,you have
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,very happy best candidate for bangalore south ...,0,0.909091,11,10,4.909091,4.264463,0.181818,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,you made country islam you are proud lets beli...,0,0.731707,41,30,4.317073,2.070196,0.195122,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199997,kannauj and allahabad will difficult for bjp t...,0,0.840000,25,21,5.760000,8.502400,0.160000,NaN,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199998,truth toh lakh aney hain aur 6000 bas choose k...,0,0.933333,15,14,5.600000,7.173333,0.000000,NaN,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df_limpo.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features.csv', index=False)